In [1]:
# Useful starting lines
%matplotlib inline

import random
from datetime import datetime

import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

# Support Vector Machines
## Classification Using SVM
Load dataset. We will re-use the CERN dataset from project 1, available from https://inclass.kaggle.com/c/epfml-project-1/data

In [3]:
cd ../

/home/mbanga/Documents/Epfl/ml_revisions


In [7]:
from supportVectorMachines.helpers import load_csv_data, predict_labels

DATA_TRAIN_PATH = 'supportVectorMachines/data/train.csv'

y, X, ids = load_csv_data(DATA_TRAIN_PATH, sub_sample=True)
print(y.shape, X.shape)

(5000,) (5000, 30)


## Prepare cost and prediction functions

In [37]:
def calculate_primal_objective(y, X, w, lambda_):
    """compute the full cost (the primal objective), that is loss plus regularizer.
    X: the full dataset matrix, shape = (num_examples, num_features)
    y: the corresponding +1 or -1 labels, shape = (num_examples)
    w: shape = (num_features)
    """
    reg = (lambda_ / 2) * w.T @ w
    sum_ = max(0, np.sum(1 - y * (X@w)))
    cost = sum_ + reg
    return cost

In [38]:
def calculate_accuracy(y, X, w):
    """compute the training accuracy on the training set (can be called for test set as well).
    X: the full dataset matrix, shape = (num_examples, num_features)
    y: the corresponding +1 or -1 labels, shape = (num_examples)
    w: shape = (num_features)
    """
    y_pred = predict_labels(w, X)
    
    num_examples = len(y)
    num_errors = np.sum((y - y_pred) / 2)
    accuracy   = 1 - num_errors / num_examples
    return accuracy

## Stochastic Gradient Descent for SVM

Compute the (stochastic) subgradient for the n-th summand of the SVM optimization objective

In [11]:
def calculate_stochastic_gradient(y, X, w, lambda_, n, num_examples):
    """compute the stochastic gradient of loss plus regularizer.
    X: the dataset matrix, shape = (num_examples, num_features)
    y: the corresponding +1 or -1 labels, shape = (num_examples)
    w: shape = (num_features)
    n: the index of the (one) datapoint we have sampled
    num_examples: N
    """
    # Be careful about the constant N (size) term!
    # The complete objective for SVM is a sum, not an average as in earlier SGD examples!
    x_n, y_n = X[n], y[n]
    
    num_features = len(w)
    grad = np.zeros((num_features,))
    
    arg  = 1 - y_n * x_n @ w
    if arg > 0:
        grad += - y_n * x_n
    
    return grad

Implement stochastic gradient descent: Pick a data point uniformly at random and update w based on the gradient for the n-th summand of the objective

In [39]:
def sgd_for_svm_demo(y, X):
    
    max_iter = 100000
    gamma = 1
    lambda_ = 0.01
    
    num_examples, num_features = X.shape
    w = np.zeros(num_features)
    
    for it in range(max_iter):
        # n = sample one data point uniformly at random data from x
        n = random.randint(0,num_examples-1)
        
        grad = calculate_stochastic_gradient(y, X, w, lambda_, n, num_examples)
        w -= gamma/(it+1) * grad
        
        if it % 10000 == 0:
            cost = calculate_primal_objective(y, X, w, lambda_)
            print("iteration={i}, cost={c}".format(i=it, c=cost))
    
    print("training accuracy = {l}".format(l=calculate_accuracy(y, X, w)))

sgd_for_svm_demo(y, X)

iteration=0, cost=2309.849171705
iteration=10000, cost=1138.852859583834
iteration=20000, cost=1042.6690947981665
iteration=30000, cost=996.3305366998293
iteration=40000, cost=962.0756926752861
iteration=50000, cost=937.3909550260676
iteration=60000, cost=917.821802136629
iteration=70000, cost=901.9443388455775
iteration=80000, cost=887.4791781397013
iteration=90000, cost=874.4599538540178
training accuracy = 0.9938


## Coordinate Descent (Ascent) for SVM

Compute the closed-form update for the n-th variable alpha, in the dual optimization problem, given alpha and the current corresponding w

In [54]:
def calculate_coordinate_update(y, X, lambda_, alpha, w, n):
    """compute a coordinate update (closed form) for coordinate n.
    X: the dataset matrix, shape = (num_examples, num_features)
    y: the corresponding +1 or -1 labels, shape = (num_examples)
    w: shape = (num_features)
    n: the coordinate to be updated
    """
    # calculate the update of coordinate at index=n.
    x_n, y_n = X[n], y[n]
    old_alpha_n = np.copy(alpha[n])
    
    # find optimal solution for dual optimization
    # solution based on my own derivations (ref. exercice book)
    X_ = X
    y_ = y
    alpha_ = alpha
    
    # trick to vectorize the calculations
    # instead of using -for- loops
    X_[n] = 0
    y_[n] = 0
    alpha_[n] = 0
    
    numerator = (1 - y_n * (alpha_.T @ y_ @ X_ @ x_n))
    denominator = (y_n**2 * x_n.T@x_n)
    new_alpha_n = numerator / denominator
    alpha[n] = new_alpha_n
    
    return w, alpha

In [51]:
def calculate_dual_objective(y, X, w, alpha, lambda_):
    """calculate the objective for the dual problem."""
    num_examples = len(y)
    left = - np.ones(num_examples).T @ alpha
    right = alpha.T @ y.T @ X @ X.T @ y @ alpha
    return left + right

In [57]:
(20000 * 500000) / 10**9

10.0

In [55]:
def coordinate_descent_for_svm_demo(y, X):
    max_iter = 100000
    lambda_ = 0.01

    num_examples, num_features = X.shape
    w = np.zeros(num_features)
    alpha = np.zeros(num_examples)
    
    for it in range(max_iter):
        # n = sample one data point uniformly at random data from x
        n = random.randint(0,num_examples-1)
        
        w, alpha = calculate_coordinate_update(y, X, lambda_, alpha, w, n)
            
        if it % 10000 == 0:
            # primal objective
            primal_value = calculate_primal_objective(y, X, w, lambda_)
            # dual objective
            dual_value = calculate_dual_objective(y, X, w, alpha, lambda_)
            # primal dual gap
            duality_gap = primal_value - dual_value
            print('iteration=%i, primal:%.5f, dual:%.5f, gap:%.5f'%(
                    it, primal_value, dual_value, duality_gap))
    print("training accuracy = {l}".format(l=calculate_accuracy(y, X, w)))

coordinate_descent_for_svm_demo(y, X)

ValueError: matmul: Input operand 0 does not have enough dimensions (has 0, gufunc core with signature (n?,k),(k,m?)->(n?,m?) requires 1)